In [ ]:
import os
import cv2
import torch
import requests
from flask import Flask, request, render_template
from ultralytics import YOLO

# Initialize YOLO model (Pre-trained YOLOv8)
model = YOLO("yolov8n.pt")

def detect_damage(video_path):
    cap = cv2.VideoCapture(video_path)
    detected_damages = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame)
        
        for result in results:
            for box in result.boxes:
                label = model.names[int(box.cls)]
                conf = float(box.conf)
                if label in ['scratch', 'dent', 'crack']:  # Adjust based on dataset labels
                    detected_damages.append({'type': label, 'confidence': conf})
                    
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return detected_damages

# Function to send detected damages to Mistral API
def predict_repair_cost(damages):
    API_KEY = "your_mistral_api_key"
    url = "https://api.mistral.ai/predict"
    
    data = {"damages": damages}
    headers = {"Authorization": f"Bearer {API_KEY}"}
    
    response = requests.post(url, json=data, headers=headers)
    return response.json()

# Flask Web App
app = Flask(__name__)
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        car_model = request.form['car_model']
        video = request.files['video']
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], video.filename)
        video.save(video_path)
        
        damages = detect_damage(video_path)
        repair_cost = predict_repair_cost(damages)
        
        return render_template("result.html", model=car_model, damages=damages, cost=repair_cost)
    
    return render_template("upload.html")

if __name__ == '__main__':
    app.run(debug=True)
